In [11]:
library(dplyr)
library(stringr)
library(shiny)
library(rvest)
library(reticulate)

source_python("LRGrades.py")

In [ ]:
# url de login do Controle Acadêmico
url <- "https://pre.ufcg.edu.br:8443/ControleAcademicoOnline"

tabelaNotas <- function(matricula, senha) {
    pgsession<-html_session(url)
    pgform<-html_form(pgsession)[[1]]  
    filled_form<-set_values(pgform, login=matricula, senha=senha)
    submit_form(pgsession, filled_form)

    urlLogada <- "https://pre.ufcg.edu.br:8443/ControleAcademicoOnline/Controlador?command=AlunoHistorico"

    page <- jump_to(pgsession, urlLogada)


    table <- page %>% read_html() %>% html_nodes(xpath = "/html/body/div[3]/div[3]/div[3]/table") %>% html_table() 
    df <- as.data.frame(table)
    df_notas <- df %>% select('Disciplina', 'Média', 'Situação')

    df_notas_base <- df_notas %>% filter(Situação != "Em Curso" , Situação != "Dispensa", !str_detect(Disciplina, "^TECC"))
    tabelaNotasFiltradas(df_notas_base)
    return(df_notas_base)
}

tabelaNotasFiltradas <- function(df_notas_base) {
    df_notas_base <- dfNotas %>% select('Disciplina', 'Média')
    df_notas_base["Matricula"] <- 0
    write.csv(df_notas_base, "temp.csv", row.names = FALSE)
    
}

matricula = "116110076"
senha = "100%mais"
shinyApp(

    ui <- fluidPage( 
        textInput("matricula", "Matrícula",width = 9),
        
        textInput("senha", "Senha",width = 20),
        
        tableOutput('table')

      ),

    server <- function(input, output) {

        output$value <- renderText({
            input$matricula
        })

        output$value <- renderText({
            input$senha
        })
        
        
        output$table <- renderTable(tabelaNotas(matricula, senha))
        output$table <- renderTable(predicao())
#         output$table <- renderTable(tabelaNotas(input$matricula, input$senha))
#         output$table <- renderTable((predicao(tabelaNotasFiltradas(input$matricula, input$senha))))

    }
)


Listening on http://127.0.0.1:4155
Submitting with '<unnamed>'
Warning message:
“Error in eval: objeto 'dfNotas' não encontrado”Stack trace (innermost first):
    131: eval
    130: eval
    129: %>%
    128: tabelaNotasFiltradas [<text>#25]
    127: tabelaNotas [<text>#20]
    126: renderTable
    125: func
    124: origRenderFunc
    123: output$table
     48: <Anonymous>
     47: do.call
     46: print.shiny.appobj
     45: print
     44: eval
     43: eval
     42: withVisible
     41: evalVis
     40: capture.output
     39: paste
     38: repr_text.default
     37: mime2repr[[mime]]
     36: withCallingHandlers
     35: doTryCatch
     34: tryCatchOne
     33: tryCatchList
     32: tryCatch
     31: FUN
     30: lapply
     29: sapply
     28: lapply
     27: unlist
     26: Filter
     25: filter_map
     24: prepare_mimebundle
     23: prepare_mimebundle_kernel
     22: value_fun
     21: withVisible
     20: withCallingHandlers
     19: doTryCatch
     18: tryCatchOne
     17